<a href="https://colab.research.google.com/github/PQuangTan/PQTan/blob/master/C_Feature_Extraction_using_Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount gg drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import library
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from imutils import paths
import os

import pandas as pd
import numpy as np

In [ ]:
# define Alexnet Architecture
class vgg19:
  def __init__(self, cs = 1000,  bp = "FC3"):
    self.classifer = cs
    self.breakPoint = bp
    self.model = None

  # setting model
  def define_model(self):
    # model = Sequential()
    # model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)))
    # model.add(BatchNormalization())
    # model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    # model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
    # model.add(BatchNormalization())
    # model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    # model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    # model.add(BatchNormalization())
    # model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    # model.add(BatchNormalization())
    # model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
    # model.add(BatchNormalization())
    # model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    # model.add(Flatten())
    # model.add(Dense(4096,activation='relu'))

    model = Sequential()

    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    # feature extraction in FC1
    if (self.breakPoint == "FC1"):
      self.model = model
      return

    model.add(Dropout(0.5))
    model.add(Dense(4096,activation='relu'))

    # feature extraction in FC2
    if (self.breakPoint == "FC2"):
      self.model = model
      return
    
    model.add(Dropout(0.5))
    model.add(Dense(self.classifer,activation='softmax'))
    self.model = model

  # predict 
  def predict(self, data):
    return self.model.predict(data)

  # get summary of model
  def summary(self):
    print(self.model.summary())

  # get model
  def get_model(self):
    return self.model

  # get/set break point
  def get_breakPoint(self):
    return self.breakPoint
  def set_breakPoint(self, bp):
    self.breakPoint = bp

  # get/set class
  def get_class(self):
    return self.classifer
  def set_class(self, c):
    self.classifer = c

In [ ]:
# get feature from model
def getFeature(imagePath, model):
  # preprocess image
  image = load_img(imagePath,target_size=(224,224))
  img_array = img_to_array(image)
  img_array = np.expand_dims(img_array, axis=0)

  # get feature 
  feature = model.predict(img_array)
  feature = feature.reshape(-1)

  return feature

In [ ]:
# feature extraction
def featureExtraction(imageDir, model):
  imagePaths = list(paths.list_images(imageDir))
  features = []
  labels = []

  for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    if label != "train":
        features.append(getFeature(imagePath,model))

        if (label == "Lines"):
            labels.append(1)
        else:
            labels.append(0)
  
  # create dataFrame
  # dataset = pd.DataFrame(
  # {
  #   "Featrues": features,
  #   "labels" : labels   
  # })
  return np.array(features),np.array(labels)

In [ ]:
# main function
if __name__ ==  "__main__":
  pathDir = '/content/drive/MyDrive/Data Nine-Dash-Lines'

  # setting model
  model = Alexnet(cs=2,bp="FC1")
  model.define_model()
  model.summary()

  # feature extraction
  # FC1
  features_1 , labels_1 = featureExtraction(pathDir, model)
  np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/FC1_features_vgg19.txt",features_1)
  np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/FC1_labels_vgg19.txt",labels_1)

  # FC2
  model.set_breakPoint(bp="FC2")
  model.define_model()
  model.summary()
  features_2, labels_2 = featureExtraction(pathDir, model)
  np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/FC2_features_vgg19.txt",features_2)
  np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/FC2_labels_vgg19.txt",labels_2)

  # # show 
  # print(dt_feature)
  # print(dataset.shape)

  # print(dataset2)
  # print(dataset.shape)

In [ ]:
# main function
pathDir = '/content/drive/MyDrive/test'

# setting model
model = vgg19(cs=2,bp="FC1")
model.define_model()
model.summary()

# feature extraction
# FC1
features_11 , labels_11 = featureExtraction(pathDir, model)
np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/test/test_FC1_features_vgg19.txt",features_11)
np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/test/test_FC1_labels_vgg19.txt",labels_11)

# FC2
model.set_breakPoint(bp="FC2")
model.define_model()
model.summary()
features_22, labels_22 = featureExtraction(pathDir, model)
np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/test/test_FC2_features_vgg19.txt",features_22)
np.savetxt("/content/drive/MyDrive/Data Nine-Dash-Lines/DataFrame/test/test_FC2_labels_vgg19.txt",labels_22)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 56, 56, 256)      